In [1]:
import numpy as np
import pandas as pd
import pickle

# 1.Data
- Load data from pickle file 
- split the data into independent and dependent 
- split to train and test set

In [2]:
data = pickle.load(open('data_face_features_emotion.pickle',mode='rb'))

In [3]:
x = np.array(data['data']) # independent vriable
y = np.array(data ['label']) # dependent variable

In [4]:
x.shape, y.shape

((1413, 1, 128), (1413,))

In [5]:
x = x.reshape(-1,128)
x.shape

(1413, 128)

In [6]:
# split the data into train and test
from sklearn.model_selection import train_test_split

In [7]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=0)

In [8]:
x_train.shape, x_test.shape, y_train.shape,y_test.shape

((1130, 128), (283, 128), (1130,), (283,))

# 2. Train Machine learning

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report,accuracy_score,f1_score

### Logistic Regression 

In [10]:
model_logistic = LogisticRegression()
model_logistic.fit(x_train,y_train) # training Logistic regression

LogisticRegression()

In [11]:
def get_report(model, x_train,y_train,x_test,y_test):  
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)

    # accuracy score 
    acc_train = accuracy_score(y_train,y_pred_train)
    acc_test = accuracy_score(y_test,y_pred_test)

    # f1 score 
    f1_score_train = f1_score(y_train,y_pred_train,average='macro')
    f1_score_test= f1_score(y_test,y_pred_test,average='macro')


    print('Accuracy Train = %0.2f'%acc_train)
    print('Accuracy Test = %0.2f'%acc_test)
    print('F1 score Train = %0.2f'%f1_score_train)
    print('F1 score Test = %0.2f'%f1_score_test)

In [12]:
get_report(model_logistic,x_train,y_train,x_test,y_test)

Accuracy Train = 0.31
Accuracy Test = 0.23
F1 score Train = 0.23
F1 score Test = 0.18


# support vector machines 

In [13]:
model_svc = SVC(probability=True)
model_svc.fit(x_train,y_train)

SVC(probability=True)

In [14]:
get_report(model_svc,x_train,y_train,x_test,y_test)

Accuracy Train = 0.42
Accuracy Test = 0.25
F1 score Train = 0.32
F1 score Test = 0.17


# Random Forest

In [15]:
model_rf = RandomForestClassifier(n_estimators=10)
model_rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=10)

In [30]:
get_report(model_rf,x_train,y_train,x_test,y_test)

Accuracy Train = 1.00
Accuracy Test = 0.45
F1 score Train = 1.00
F1 score Test = 0.46


# Voting Classifier

In [17]:
model_voting= VotingClassifier(estimators=[
    ('logistic',LogisticRegression()),
    ('svm',SVC(probability=True)),
    ('rf',RandomForestClassifier())
], voting='soft',weights=[2,3,1])

In [18]:
model_voting.fit(x_train,y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [31]:
get_report(model_voting,x_train,y_train,x_test,y_test)

Accuracy Train = 0.84
Accuracy Test = 0.40
F1 score Train = 0.85
F1 score Test = 0.38


# Parameter Tuning

In [35]:
from sklearn.model_selection import GridSearchCV

In [36]:
model_grid = GridSearchCV(model_voting,
                         param_grid={
                             'svm__C':[3,5,7,10],
                             'svm__gamma':[0.1,0.3,0.5],
                             'rf__n_estimators':[5,10,20],
                             'rf__max_depth':[3,5,7],
                             'voting':['soft','hard']
                         },scoring='accuracy',cv=3,n_jobs=1,verbose=2)

In [37]:
model_grid.fit(x_train,y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.5s
[

[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.8s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3

limit_output extension: Maximum message size of 10000 exceeded with 10089 characters

In [ ]:
model_best_estimator = model_grid.best_estimator_

In [ ]:
model_grid.best_score_

# 4. save model

In [46]:
pickle.dump(model_best_estimator,open('./models/machinelearning_face_person_identity.pkl',mode='wb'))